In [1]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

import numpy as np

import pydrake.all
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix, Simulator, FixedOffsetFrame
    )

from panda_station import *

In [7]:
builder = DiagramBuilder()
station = builder.AddSystem(PandaStation())
station.setup_from_file("directives/three_tables.yaml")
station.add_panda_with_hand()
plant, scene_graph = station.get_plant_and_scene_graph()

#station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))
station.add_model_from_file(find_resource("models/manipulands/sdf/dumbbell.sdf"), RigidTransform(RotationMatrix(), [0.8, 0, 0.2]),
                           main_body_name = "base_link")
frame = FixedOffsetFrame("foam_brick", plant.world_frame(), RigidTransform(RotationMatrix(), [0.6, 0, 0.25]))
plant.AddFrame(frame)

parser = Parser(plant)
brick = parser.AddModelFromFile(find_resource("models/manipulands/sdf/foam_brick.sdf"), "foam_brick")
plant.WeldFrames(frame, plant.GetFrameByName("base_link", brick), RigidTransform())


station.finalize()
print(station.get_directive())


meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()
diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)
station.GetInputPort("panda_position").FixValue(station_context, np.array([np.pi/4, 0.1, 0, -1, 0, 1.5, 0]))
station.GetInputPort("hand_position").FixValue(station_context, [0.08])

meshcat.start_recording()
simulator.AdvanceTo(0.2)
simulator_context = simulator.get_context()
plant_context = plant.GetMyContextFromRoot(simulator_context)
station_context = station.GetMyContextFromRoot(simulator_context)


print("PARSING START POSES:")
print(parse_start_poses(station, station_context))
print("PARSING INITIAL CONFIG")
print(parse_initial_config(station, station_context))
print("PARSING TABLES")
print(parse_tables(station))
print("CREATING WELDED STATION")
welded_station, body_infos = create_welded_station(station, station_context)


frame.SetPoseInBodyFrame(plant_context, RigidTransform(RotationMatrix(), [0.6, 0.5, 0.2]))
simulator.AdvanceTo(2.0)
meshcat.stop_recording()
meshcat.publish_recording()

/home/agrobenj/drake-tamp/panda_station/directives/three_tables.yaml
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
PARSING START POSES:
{'added_model_0': RigidTransform(
  R=RotationMatrix([
    [0.999999998074673, -5.329757998026651e-05, 3.178084453479318e-05],
    [5.329742069706724e-05, 0.9999999985671286, 5.0127505419925114e-06],
    [-3.178111165672825e-05, -5.011056695300048e-06, 0.9999999994824251],
  ]),
  p=[0.8000033267394582, 2.187963729147095e-07, 0.15925366217150516],
)}
PARSING INITIAL CONFIG
{'panda': array([  4.68669110e-01,   9.99952726e-02,  -3.52246680e-05,
        -1.08070244e+00,   1.31657919e-04,   1.54057927e+00,
         7.07745213e-04])}
PARSING TABLES
['table', 'table_square', 'table_round']
CREATING WELDED STATION


In [8]:
for name, info in body_infos.items():
    print(name)
    print(info)

added_model_0

        main body index: BodyIndex(16)
        offset_frame: <FixedOffsetFrame_[float] name='frame_added_model_0' index=43 model_instance=0> 
        cylinder <FixedOffsetFrame_[float] name='frame_added_model_0_base_link_0' index=44 model_instance=7>
cylinder <FixedOffsetFrame_[float] name='frame_added_model_0_bottom_weight_0' index=45 model_instance=7>
cylinder <FixedOffsetFrame_[float] name='frame_added_model_0_top_weight_0' index=46 model_instance=7>

table

        main body index: BodyIndex(1)
        offset_frame: None 
        box <FixedOffsetFrame_[float] name='frame_table_base_link_0' index=47 model_instance=2>

table_square

        main body index: BodyIndex(2)
        offset_frame: None 
        box <FixedOffsetFrame_[float] name='frame_table_square_base_link_0' index=48 model_instance=3>

table_round

        main body index: BodyIndex(3)
        offset_frame: None 
        cylinder <FixedOffsetFrame_[float] name='frame_table_round_base_link_0' index=49 mode

In [9]:
welded_station, body_infos = create_welded_station(station, station_context, holding_index = station.object_infos["added_model_0"].main_body_index)
for name, info in body_infos.items():
    print(name)
    print(info)

added_model_0

        main body index: BodyIndex(16)
        offset_frame: None 
        cylinder <FixedOffsetFrame_[float] name='frame_added_model_0_base_link_0' index=43 model_instance=7>
cylinder <FixedOffsetFrame_[float] name='frame_added_model_0_bottom_weight_0' index=44 model_instance=7>
cylinder <FixedOffsetFrame_[float] name='frame_added_model_0_top_weight_0' index=45 model_instance=7>

table

        main body index: BodyIndex(1)
        offset_frame: None 
        box <FixedOffsetFrame_[float] name='frame_table_base_link_0' index=46 model_instance=2>

table_square

        main body index: BodyIndex(2)
        offset_frame: None 
        box <FixedOffsetFrame_[float] name='frame_table_square_base_link_0' index=47 model_instance=3>

table_round

        main body index: BodyIndex(3)
        offset_frame: None 
        cylinder <FixedOffsetFrame_[float] name='frame_table_round_base_link_0' index=48 model_instance=4>

